In [1]:
import foolbox as fb
import art
import tensorflow as tf
import numpy as np
import joblib

/home/bossantm/anaconda3/envs/SDNIOT-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/bossantm/anaconda3/envs/SDNIOT-env/lib/python3.10/site-packages/art/estimators/certification/__init__.py:30: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")
2025-06-13 14:21:56.403837: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
X_train = joblib.load('../../preprocessed_data/CICIDS/X_train.joblib')
X_test = joblib.load('../../preprocessed_data/CICIDS/X_test.joblib')
Y_train = joblib.load('../../preprocessed_data/CICIDS/Y_train.joblib')
Y_test = joblib.load('../../preprocessed_data/CICIDS/Y_test.joblib')

In [3]:
from tensorflow.keras.models import load_model

cnn_loaded = load_model('../../models/cnn_CICIDS2017_model.keras')
lstm_loaded = load_model('../../models/lstm_CICIDS2017_model.keras')
ae_lstm_loaded = load_model('../../models/ae-lstm_CICIDS2017_model.keras')



# import pickle

# with open("../../models/cnn_CICIDS2017.pkl", "rb") as f:
#     cnn_loaded = pickle.load(f)

# with open("../../models/lstm_CICIDS2017.pkl", "rb") as f:
#     lstm_loaded = pickle.load(f)

# with open("../../models/ae-lstm_CICIDS2017.pkl", "rb") as f:
#     ae_lstm_loaded = pickle.load(f)

In [ ]:
from art.attacks.evasion import FeatureAdversariesTensorFlowV2, HopSkipJump, BoundaryAttack
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_mnist

classifier = TensorFlowV2Classifier(
    model=cnn_loaded,
    nb_classes=2,
    input_shape=(X_test.shape[1], X_test.shape[2]),
    loss_object=tf.keras.losses.BinaryCrossentropy()
)

attack_hsj = HopSkipJump(classifier=classifier, norm=2,max_iter=5000, max_eval=10000, init_eval=100, init_size=100)
adv_hopskipjump = attack_hsj.generate(x=X_test[:1000])
attack_boundary = BoundaryAttack(estimator=classifier)
adv_boundary = attack_boundary.generate(x=X_test[:1000], y=Y_test[:1000])

# # original predictions
# preds_orig = classifier.predict(X_test[:20])

# # HopSkipJump predictions
# preds_adv_hsj = classifier.predict(adv_hopskipjump)

# # BoundaryAttack predictions
# preds_adv_boundary = classifier.predict(adv_boundary)

# print("Original predictions       :", np.argmax(preds_orig, axis=1))
# print("After HopSkipJump attack   :", np.argmax(preds_adv_hsj, axis=1))
# print("After BoundaryAttack       :", np.argmax(preds_adv_boundary, axis=1))

# Original predictions
preds_orig = classifier.predict(X_test[:1000])
labels_orig = (preds_orig > 0.5).astype(int).flatten()

# HopSkipJump predictions
preds_adv_hsj = classifier.predict(adv_hopskipjump)
labels_adv_hsj = (preds_adv_hsj > 0.5).astype(int).flatten()

# BoundaryAttack predictions
preds_adv_boundary = classifier.predict(adv_boundary)
labels_adv_boundary = (preds_adv_boundary > 0.5).astype(int).flatten()

# Print summary counts instead of raw arrays
from collections import Counter
print("=== Prediction Counts ===")
print("Original        :", Counter(labels_orig))
print("HopSkipJump     :", Counter(labels_adv_hsj))
print("BoundaryAttack  :", Counter(labels_adv_boundary))

# Optional: show exact class change counts
print()
print("Changed by HopSkipJump    :", sum(labels_orig != labels_adv_hsj))
print("Changed by BoundaryAttack :", sum(labels_orig != labels_adv_boundary))

Boundary attack: 100%|██████████| 20/20 [00:00<00:00, 109.41it/s]


=== Prediction Counts ===
Original        : Counter({0: 19, 1: 1})
HopSkipJump     : Counter({0: 19, 1: 1})
BoundaryAttack  : Counter({0: 19, 1: 1})

Changed by HopSkipJump    : 0
Changed by BoundaryAttack : 0
